In [1]:
import numpy as np

## Computer Simulation

We use the following three factors to model the percent of EV car sales in the U.S: Price (P), Charging Stations (C) and Range (R). We write the following model:


$$\mathrm{Rating} = w_p \cdot f_p(P) + w_c \cdot f_c(C) + w_r \cdot f_r(R)$$

**Rating**: We interpret a rating as the probability that a particular person would buy an EV over a regular car, and take the average of these to get a prediction for proportion of EVs bought.

**weights**: Each person has a different set of preferences, $w_p$, $w_c$ and $w_r$. We sample these as:
* $w_p \sim N(0.6, 0.1)$
* $w_c \sim N(0.05 + \frac{1 - w_p}{2}, 0.01^2)$
* $w_r = 1 - w_p - w_c$

**f**: this is a function that scores each factor between 0 and 1.

In [2]:
# seed for reported results
np.random.seed(157)

In [3]:
def sample_weights(n=1):
    w_p = np.random.normal(loc=0.6, scale=0.1, size=n)
    w_c = (1 - w_p) / 2 + np.random.normal(loc=0.05, scale=0.01, size=n)
    w_r = 1 - w_p - w_c
    return w_p, w_c, w_r

def sample_P(n=1):
    return np.random.normal(loc=0.95, scale=0.2, size=n)
    
def sample_C(n=1):
    return np.random.normal(loc=300000, scale=75000, size=n)
    
def sample_R(n=1):
    return np.random.normal(loc=400, scale=70, size=n)

def f_p(P):
    return 1 - 1 / (1 + np.exp(-6 * (P - 1)))

def f_c(C):
    sample = 2 * (10 ** -6) * C
    return np.clip(sample, 0, 1)

def f_r(R):
    sample = 0.04 * np.sqrt(np.clip(R - 100, a_min=0, a_max=None))
    return np.clip(sample, 0, 1)

def sample_rating(n):
    w_p, w_c, w_r = sample_weights(n)
    P, C, R = sample_P(n), sample_C(n), sample_R(n)
    return w_p * f_p(P) + w_c * f_c(C) + w_r * f_r(R)

In [4]:
num_samples = 100000
ratings = sample_rating(num_samples)
print(f"Average: {np.mean(ratings)}")
print(f"Median: {np.median(ratings)}")
print(f"10% percentile: {np.percentile(ratings, 10)}")
print(f"90% percentile: {np.percentile(ratings, 90)}")

Average: 0.5877430710881961
Median: 0.5979084379048134
10% percentile: 0.383074836451778
90% percentile: 0.7760753789442462


### Alternative Weight Sampler

Samples weights from normal distributions (with different means), then normalizes result so that they sum to 0

In [5]:
def sample_weights_alt(n=1):
    w_p = np.random.normal(loc=0.7, scale=0.1, size=n)
    w_c = np.random.normal(loc=0.5, scale=0.1, size=n)
    w_r = np.random.normal(loc=0.3, scale=0.1, size=n)
    w_p, w_c, w_r = np.vstack([w_p, w_c, w_r]) / np.sum([w_p, w_c, w_r], axis=0)
    return w_p, w_c, w_r

In [6]:
num_samples = 100000
w_p, w_c, w_r = sample_weights_alt(num_samples)
P, C, R = sample_P(num_samples), sample_C(num_samples), sample_R(num_samples)
ratings = w_p * f_p(P) + w_c * f_c(C) + w_r * f_r(R)

print(f"Average: {np.mean(ratings)}")
print(f"Median: {np.median(ratings)}")
print(f"10% percentile: {np.percentile(ratings, 10)}")
print(f"90% percentile: {np.percentile(ratings, 90)}")

Average: 0.5974017869915615
Median: 0.6038190009435745
10% percentile: 0.4290335784881638
90% percentile: 0.7548683421226783
